### TP4: univariate GANs for financial time series 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


In [ ]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

    

In [ ]:
import yfinance
SPY=yfinance.Ticker("^GSPC")
mydata=SPY.history(start="1900-01-01",auto_adjust=True)

In [ ]:
rets=np.diff(np.log(mydata['Close']))


In [ ]:
plt.plot(rets)

In [ ]:
T=252

dim_noise=????   # as in the paper

In [ ]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Input, Dense, Activation, Reshape,Flatten, Dropout, Lambda, RepeatVector,Concatenate
from tensorflow.keras.layers import Add,Multiply
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import UpSampling1D, Conv1D
from tensorflow.keras.layers import LeakyReLU

def generator_model_mlp(dim_conds,dim_noise,use_bias=False):
    input_conds = Input(shape=(dim_conds,))
    input_eta = Input(shape=(dim_noise,))
    
    input_merged = Concatenate()([input_conds, input_eta])
    
#   define here model, an object in the functional way:    

    
    output_merged = Concatenate()(###### FILL ME ######) 
   
    return Model([input_conds,input_eta],output_merged)    # syntax: Model(inputs,outputs)
    

def discriminator_model_mlp(T,use_bias=True,activation="relu"):
    
#### implement here a discriminator ####
    return discriminator

In [ ]:
generator=generator_model_mlp(T-1,dim_noise,use_bias=True)

discriminator=discriminator_model_mlp(T,use_bias=True)


INIT_LR =?==# as in the paper

opt_discriminator = ????    # Check if the paper uses the same one
discriminator.compile(loss='binary_crossentropy', optimizer=opt_discriminator)


In [ ]:
discriminator.trainable = False

ganInput = [Input(shape=(T-1,)),Input(shape=(dim_noise,))]       # two types of inputs
ganOutput = discriminator(generator(ganInput))
gan = Model(ganInput, ganOutput)
# compile the GAN
opt_gan = ????
gan.compile(loss="binary_crossentropy", optimizer=opt_gan)

In [ ]:
from sklearn.utils import shuffle
from sklearn.preprocessing import scale

num_epochs=2000
batch_size=128
batch_size_RMSE=5000            # number of samples to compute the RMSE

num_batches_per_epoch=int(len(rets)/batch_size)
losses=[]


In [ ]:
for myepoch in range(num_epochs):
    print("epoch "+str(myepoch+1)+"/"+str(num_epochs))
    for batch in range(num_batches_per_epoch//10):
        rets_sample=[rets[range(i,i+T)] for i in np.random.choice(len(rets)-T,size=batch_size)]
        rets_sample=np.array(rets_sample)         # list -> 2d-array
        rets_sample=scale(rets_sample,axis=1)

        conds=rets_sample[:,:-1]
        etas=np.random.normal(size=(batch_size,dim_noise))     # generates the feature noise

        preds=generator.predict(##### PLEASE FILL ME #####)
      
        X=np.concatenate((preds,rets_sample))
        y=np.append(np.full(batch_size,0),np.full(batch_size,1))
        (X,y)=shuffle(X,y)   # otherwise the discriminator learns the order

        X=np.expand_dims(X, axis=2)
        y=np.expand_dims(y,1)

        discriminator_loss=discriminator.train_on_batch(X, y)    # improves discriminator classifiation abilities
                                                                 # this temporarilly unfreezes the discr. weights

            
        rets_sample=[rets[range(i,i+T)] for i in np.random.choice(len(rets)-T,size=batch_size)]
        rets_sample=np.array(rets_sample)         # list -> 2d-array
        rets_sample=scale(rets_sample,axis=1)

        conds=rets_sample[:,:-1]
        etas=np.random.normal(size=(batch_size,dim_noise))     

        y_fake=np.full(etas.shape[0],1)

        gan_loss = gan.train_on_batch(##### PLEASE FILL ME #####,y_fake)               # trains the generator
    
    
    rets_sample=[rets[range(i,i+T)] for i in np.random.choice(len(rets)-T,size=batch_size_RMSE)]
    rets_sample=np.array(rets_sample)         # list -> 2d-array
    rets_sample=scale(rets_sample,axis=1)

    conds=rets_sample[:,:-1]
    etas=np.random.normal(size=(batch_size_RMSE,dim_noise))     # replace the last column by noise


    RMSE=np.sqrt(np.mean((generator.predict(##### PLEASE FILL ME #####)[:,-1]-rets_sample[:,-1])**2))
    
    new_losses=[gan_loss,discriminator_loss,RMSE]
    print(new_losses)
    losses.append(np.array(new_losses))
    